In [42]:
import os
import json
import pandas as pd
import traceback
import warnings
warnings.filterwarnings("ignore")

In [10]:
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
load_dotenv()

True

In [11]:
KEY = os.getenv("OPENAI_API_KEY")

In [14]:
llm = ChatOpenAI(openai_api_key=KEY,model_name='gpt-3.5-turbo',temperature=0.3)

In [16]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [17]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [18]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [19]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [20]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)


In [21]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [22]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)


In [23]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)


In [24]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [25]:
file_path=r"/Users/sbhardwa/Documents/Learning/Machine Learning & AI/Gen AI/mcqgen/data.txt"

In [26]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [27]:
print(TEXT)

Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other scientists, bio

In [28]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [30]:
NUMBER=5 
SUBJECT="biology"
TONE="simple"

In [31]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )

/Users/sbhardwa/Documents/Learning/Machine Learning & AI/Gen AI/mcqgen/mcq_venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each define

In [32]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:1479
Prompt Tokens:1067
Completion Tokens:412
Total Cost:0.0024244999999999996


In [33]:
quiz=response.get("quiz")


In [35]:
quiz=json.loads(quiz)


In [36]:
quiz

{'1': {'mcq': 'What is the scientific study of life called?',
  'options': {'a': 'Chemistry',
   'b': 'Biology',
   'c': 'Physics',
   'd': 'Geology'},
  'correct': 'b'},
 '2': {'mcq': 'Which theme in biology explains the unity and diversity of life?',
  'options': {'a': 'Evolution',
   'b': 'Photosynthesis',
   'c': 'Cellular respiration',
   'd': 'Mitosis'},
  'correct': 'a'},
 '3': {'mcq': 'What allows organisms to move, grow, and reproduce?',
  'options': {'a': 'Energy processing',
   'b': 'Genetic mutation',
   'c': 'Cell division',
   'd': 'Photosynthesis'},
  'correct': 'a'},
 '4': {'mcq': 'What is the basic unit of life that processes hereditary information?',
  'options': {'a': 'Tissue', 'b': 'Organ', 'c': 'Cell', 'd': 'Organism'},
  'correct': 'c'},
 '5': {'mcq': 'When did life on Earth emerge?',
  'options': {'a': '1 million years ago',
   'b': '3.7 billion years ago',
   'c': '10,000 years ago',
   'd': '500 million years ago'},
  'correct': 'b'}}

In [47]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

KeyError: 'mcq'

In [38]:
quiz_table_data

[{'MCQ': 'What is the scientific study of life called?',
  'Choices': 'a: Chemistry | b: Biology | c: Physics | d: Geology',
  'Correct': 'b'},
 {'MCQ': 'Which theme in biology explains the unity and diversity of life?',
  'Choices': 'a: Evolution | b: Photosynthesis | c: Cellular respiration | d: Mitosis',
  'Correct': 'a'},
 {'MCQ': 'What allows organisms to move, grow, and reproduce?',
  'Choices': 'a: Energy processing | b: Genetic mutation | c: Cell division | d: Photosynthesis',
  'Correct': 'a'},
 {'MCQ': 'What is the basic unit of life that processes hereditary information?',
  'Choices': 'a: Tissue | b: Organ | c: Cell | d: Organism',
  'Correct': 'c'},
 {'MCQ': 'When did life on Earth emerge?',
  'Choices': 'a: 1 million years ago | b: 3.7 billion years ago | c: 10,000 years ago | d: 500 million years ago',
  'Correct': 'b'}]

In [40]:
quiz=pd.DataFrame(quiz_table_data)
quiz


,MCQ,Choices,Correct
0,What is the scientific study of life called?,a: Chemistry | b: Biology | c: Physics | d: Ge...,b
1,Which theme in biology explains the unity and ...,a: Evolution | b: Photosynthesis | c: Cellular...,a
2,"What allows organisms to move, grow, and repro...",a: Energy processing | b: Genetic mutation | c...,a
3,What is the basic unit of life that processes ...,a: Tissue | b: Organ | c: Cell | d: Organism,c
4,When did life on Earth emerge?,a: 1 million years ago | b: 3.7 billion years ...,b


In [41]:
quiz.to_csv("machinelearning.csv",index=False)


In [43]:
quiz_str = {
"1": {
"mcq": "What is the main theme that explains the unity and diversity of life?",
"options": {
"a": "Evolution",
"b": "Energy processing",
"c": "Cell structure",
"d": "Regulation of internal environments"
},
"correct": "a"
},
"2": {
"mcq": "What is the scientific study of life known as?",
"options": {
"a": "Chemistry",
"b": "Biology",
"c": "Physics",
"d": "Geology"
},
"correct": "b"
},
"3": {
"mcq": "What is the process that allows organisms to move, grow, and reproduce?",
"options": {
"a": "Evolution",
"b": "Energy processing",
"c": "Cell division",
"d": "Photosynthesis"
},
"correct": "b"
}
}

In [45]:
quiz_dict = json.loads(str(quiz_str))
quiz_table_data = []

for key, value in quiz_dict.items():
    mcq = value["mcq"]
    options = " || ".join(
        [
            f"{option}-> {option_value}" for option, option_value in value["options"].items()
        ]
    )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)